<a href="https://colab.research.google.com/github/mahmoudmagdyhassan/Mahmoud-/blob/main/customer%20segmentation%20analysis%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import datetime
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

random_state = 50

In [ ]:
data = pd.read_csv("/content/marketing_campaign.csv", sep="\t")
data.head(3)

In [ ]:
data.shape

In [ ]:
def d_types_report(df):
    columns=[]
    d_types=[]
    uniques=[]
    n_uniques=[]
    null_values=[]
    null_values_percentage=[]
    rows = df.shape[0]

    for i in df.columns:
        columns.append(i)
        d_types.append(df[i].dtypes)
        uniques.append(df[i].unique()[:5])
        n_uniques.append(df[i].nunique())
        null_values.append(df[i].isna().sum())
        null_values_percentage.append(null_values[-1] * 100 / rows)

    return pd.DataFrame({"Columns": columns, "Data_Types": d_types, "Unique_values": uniques, "N_Uniques": n_uniques,  "Null_Values": null_values, "Null_Values_percentage": null_values_percentage})

d_types_report(data)

In [ ]:
#remove null value
data = data.dropna()

In [ ]:
data.drop(columns=['ID'],inplace=True)
data["Dt_Customer"] = pd.to_datetime(data["Dt_Customer"]).dt.date
data.Dt_Customer.min(),data.Dt_Customer.max()

In [ ]:
data.describe().T

In [ ]:
for column in data:
    plt.figure(figsize=(8, 6))

    if data[column].dtype == 'int64' or data[column].dtype == 'float64':
        # Numerical columns
        plt.title(f'Distribution of {column}', fontsize=16)
        #print(data[column].value_counts(normalize=True))
        sns.histplot(data=data[column],kde=True)
        plt.xlabel(column, fontsize=14)
        plt.ylabel('Frequency', fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()

In [ ]:
for col in ['Education', 'Marital_Status']:
    plt.figure(figsize=(8, 6))
    plt.title(f'Distribution of {col}', fontsize=16)
    print(data[col].value_counts(normalize=True))
    sns.histplot(data=data[col])
    plt.xlabel(col, fontsize=14)
    plt.ylabel('Frequency', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()
    fig = px.pie(data_frame = data , names = col)
    fig.show()

#feature engineering

```
# This is formatted as code
```



In [ ]:
data["Age"] = 2023 - data["Year_Birth"]
# Total spending on various item
data["Spent"] = data["MntWines"] + data["MntFruits"] + data["MntMeatProducts"] + data["MntFishProducts"] + data["MntSweetProducts"] + data["MntGoldProds"]
data.head(3)

In [ ]:
data= data.drop(['Year_Birth','Dt_Customer','MntWines',"MntFruits","MntMeatProducts",'MntFishProducts',
'MntSweetProducts','MntGoldProds'], axis=1)

In [ ]:
data.head(10)

#**outliar**

In [ ]:
for column in ['Age', 'Spent','Income']:
    plt.figure(figsize=(8, 6))

    if data[column].dtype == 'int64' or data[column].dtype == 'float64':
        # Numerical columns
        plt.title(f'Distribution of {column}', fontsize=16)
        sns.boxplot(data=data[column])
        plt.xlabel(column, fontsize=14)
        plt.ylabel('Frequency', fontsize=14)
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()

In [ ]:
# Removing outliers for "Age" and "Income" 'spent
data_new = data[(data["Age"] < 90) & (data["Income"] < 100000)
&(data["Spent"] < 2500)]
print("The total number of data-points after removing the outliers are:", len(data_new))

In [ ]:
cat_cols = data_new.select_dtypes(include='object').columns.tolist()
num_cols =data_new.select_dtypes(exclude='object').columns.tolist()
num_cols

In [ ]:
data_new.corr()

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(data.corr(),annot=True)

In [ ]:
!pip install category_encoders

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler ,PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from category_encoders import BinaryEncoder

In [ ]:
X= data_new[['Income', 'Spent']]
X.shape
#Spent

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X= scaler.fit_transform(X)
#X = pd.DataFrame(Xpr, columns=X.columns)
X

In [ ]:
# Within-cluster Sum of Squares (Inertia)
inertia=[]
k_range= range(2,11)

for i in k_range:
    kmeans_i = KMeans(n_clusters= i, init='k-means++', n_init='auto', random_state=0)
    kmeans_i.fit(X)
    inertia.append(kmeans_i.inertia_)

# Visualizing the ELBOW method to get the optimal value of K
sns.lineplot(x=k_range, y=inertia)
plt.title('The Elbow Method')
plt.xlabel('No of clusters "k"')
plt.ylabel('Inertia')

# Annotate arrow
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_scores=[]
k_range= range(2,11)

for i in k_range:
    kmeans = KMeans(n_clusters= i, init='k-means++', n_init='auto', random_state=0)
    kmeans.fit(X)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_, metric='euclidean'))

# Visualizing the silhouette_scores method to get the optimal value of K
sns.lineplot(x=k_range, y=silhouette_scores)
plt.title('The silhouette_scores Method')
plt.xlabel('No of clusters "k"')
plt.ylabel('silhouette_scores')

# Annotate arrow
plt.show()


In [ ]:
from sklearn.cluster import KMeans

km5 = KMeans(n_clusters=3, init='k-means++', n_init='auto', random_state=0)
y_pred = km5.fit_predict(X)
#Visualizing all the clusters
plt.figure(figsize=(7,5))
color_dict = {0:'yellow', 1: 'red', 2: 'blue'}
c=pd.DataFrame(y_pred)[0].map(color_dict)
plt.scatter(x = X[:,0],y = X[:,1],c = c,cmap = 'rainbow',edgecolor = 'black')
plt.scatter(km5.cluster_centers_[:, 0], km5.cluster_centers_[:, 1], s = 70, c = 'black', label = 'Centroids')


plt.title('Clusters of customers')
plt.xlabel('Annual Income (k$)')
plt.ylabel('spent')
plt.legend()
plt.show()

In [ ]:
#Cluster 0 :hig(high income,high spent) yelow
#Cluster 2:mid(mid income,mid spent) blue
#Cluster 1 :low(low income,low spent) red

In [ ]:
data_new['cluster'] = y_pred

In [ ]:

data_new['cluster'] = data_new['cluster'].replace([1, 2,0], ['low','Intermittent','Best'])

data_new.head()

In [ ]:
data_new.head(20)

In [ ]:
sns.countplot(data_new, x="cluster").set(title="Distribution of clusters")
plt.show()

In [ ]:
sns.boxplot(x='cluster', y='Age', data=data_new)
plt.show()

In [ ]:
data_new.cluster.value_counts()

In [ ]:
sns.boxplot(x='cluster', y='Age', data=data_new)
plt.show()

Customer Age vs. Clusters:
"Intermittent High Spenders" and "Best Customers" are the oldest in terms of age, further emphasizing their long-term engagement with the store.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))
sns.countplot(ax=axes[0], x='Education', hue='cluster', data=data_new)
sns.countplot(ax=axes[1], x='Marital_Status', hue='cluster', data=data_new)
plt.show()

 "Education" and "Marital_Status" doesnt have a significant effect on the clusters

In [ ]:
sns.countplot(x='Kidhome', hue='cluster', data=data_new)
plt.show()

"Best" customers are populated in the family of 0 kidhome





"low" customers are populated in the family of 1 kidhome

In [ ]:
sns.countplot(x='Teenhome', hue='cluster', data=data_new)


Best" customers are populated in the family of 0 teenhome



"low" customers are populated in the family of 1 teenhome

In [ ]:
sns.countplot(x='NumDealsPurchases', hue='cluster', data=data_new)

in NumDealsPurchases feature, every cluster has a similar pattern. The majority of customer only made 1 purchase with a deal. But there is a small percentage of customer that made 2 purchases with a deal. And there is a very small percentage of customer that made 3 purchases and above with a deal.

In [ ]:
sns.countplot(x='NumWebPurchases', hue='cluster', data=data_new)

low group___
the majority of them made 3 purchases

In [ ]:
sns.countplot(x='NumCatalogPurchases', hue='cluster', data=data_new)

low group___ the majority of them made 1 purchases



But for  Intermittent  the majority of them made 2 purchases and above.


But for  Best the majority of them made 6 purchases and above.




In [ ]:
sns.countplot(x='NumStorePurchases', hue='cluster', data=data_new)

**Best Customers¶**




Recognize and reward "Best Customers" for their long-term loyalty



Consider offering them exclusive privileges, such as early access to sales, premium customer support, or personalized gifts on special occasions.






Create a dedicated loyalty program for this segment with tiered benefits to encourage them to maintain their high spending levels, such as free shipping, exclusive discounts, or free gifts.




Focus on building long-term relationships rather than just transactions. Prioritize customer satisfaction and consider periodic check-ins or loyalty surveys to gauge their evolving preferences.




**Intermittent High Spenders**



*Implement a reminder system to notify them about ongoing promotions or limited-time deals.



Create bundles or packages that encourage bulk purchasing, offering them cost savings for buying in larger quantities.




Tailor the offers to align with the preferences and needs of older customers.





**low**

Focus on reactivating these customers by sending win-back campaigns with attractive incentives, such as special discounts or offers for returning customers.




Show them ads featuring items they've viewed or wishlisted but didn't purchase.





Showcase positive reviews and testimonials from satisfied customers, as social proof can influence purchasing decisions